<a href="https://colab.research.google.com/github/paritoshkc/TextAnalytics/blob/master/Review_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import lxml.html as lh
import pandas as pd
import re



# Get movie name and year list from wikipedia 
url_wiki='https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films'
to_year=2010

#Create a handle, page, to handle the contents of the website
page = requests.get(url_wiki).text
soup= BeautifulSoup(page, "lxml")

My_table = soup.find('table',{'class':'wikitable sortable'})
links = My_table.find_all('a')

# for future use
name=[]
year=[]


# used for constrcution of urls
url_front = 'https://www.rottentomatoes.com/m/'
url_back_critics='/reviews'
url_back_audi='/reviews?type=user'
url_back_critics_page='?type=&sort=&page='

url_critics=[]
url_audi=[]
url_mid=[]


i=0
texts=[]
for link in links:
  texts.append(link.text.split('\n'))
for text in texts:
  if text[0]=='[1]':
    continue
  if i%2==0:
    text_string=text[0].replace(' ','_').lower()
    name.append(text_string)
  else:
    year.append(text[0])
    if int(text[0])<to_year:
      break
  i+=1

zippedList =  list(zip(name,year))

for n,y in zippedList:
  url_mid.append(n+'_'+y)

for mid in url_mid:
  url_critics.append(url_front+mid+url_back_critics)
  url_audi.append(url_front+mid+url_back_audi)




In [2]:
url_critics

['https://www.rottentomatoes.com/m/parasite_2019/reviews',
 'https://www.rottentomatoes.com/m/ford_v_ferrari_2019/reviews',
 "https://www.rottentomatoes.com/m/learning_to_skateboard_in_a_warzone_(if_you're_a_girl)_2019/reviews",
 "https://www.rottentomatoes.com/m/the_neighbors'_window_2019/reviews",
 'https://www.rottentomatoes.com/m/little_women_2019/reviews',
 'https://www.rottentomatoes.com/m/marriage_story_2019/reviews',
 'https://www.rottentomatoes.com/m/jojo_rabbit_2019/reviews',
 'https://www.rottentomatoes.com/m/toy_story_4_2019/reviews',
 'https://www.rottentomatoes.com/m/joker_2019/reviews',
 'https://www.rottentomatoes.com/m/once_upon_a_time_in_hollywood_2019/reviews',
 'https://www.rottentomatoes.com/m/1917_2019/reviews',
 'https://www.rottentomatoes.com/m/judy_2019/reviews',
 'https://www.rottentomatoes.com/m/bombshell_2019/reviews',
 'https://www.rottentomatoes.com/m/rocketman_2019/reviews',
 'https://www.rottentomatoes.com/m/american_factory_2019/reviews',
 'https://www.

In [3]:

user_review=[]
critic_review=[]

def get_user_reviews(url):
    ur_review=[]
  
    print(url)
    response = requests.get(url)
    if response.status_code ==200:
      soup = BeautifulSoup(response.text, "html.parser")
      # audience reviews
      audience_reviews=soup.findAll('p', attrs={'class': 'audience-reviews__review--mobile js-review-text clamp clamp-4 js-clamp'})
      if len(audience_reviews)!=0 :
        for reviews in audience_reviews:
          # print(reviews.text)
          ur_review.append(reviews.text+' %% ')
    return ur_review


def get_ctric_reviews(url):

  response = requests.get(url)
 
  crtc_review=[]
  print(url)
  soup = BeautifulSoup(response.text, "html.parser")
  page_no=soup.findAll('span', attrs={'class': 'pageInfo'})
  tes=0
  if len(page_no)!=0:
    tes=0
    num=int(page_no[0].text[-2:])
    while(num!=0):
      # print(num)
      # print(len(crtc_review))
      response = requests.get(url+url_back_critics_page+str(num))
      if response.status_code ==200:
        c_reviews=soup.findAll('div', attrs={'class': 'the_review'})
        num-=1
        
        if len(c_reviews)!=0 :
          for reviews in c_reviews:
            tes+=1
            crtc_review.append(reviews.text.replace('                                   ','').replace('\n','').replace('\n','').replace('                                ','')+'$$' )
  # print('total reviews for '+url+str(len(crtc_review)))#
  return crtc_review

  
def get_review_year(year):
  for user_url,critic_url in zip(url_audi,url_critics):
    if year in user_url:
      user_review.append(get_ctric_reviews(critic_url))
      critic_review.append(get_user_reviews(user_url))

get_review_year('2019')

zippedList =  list(zip(url_mid,url_critics,url_audi,user_review,critic_review))
dfObj = pd.DataFrame(zippedList, columns = ['Name' , 'url_critics','url_audi','user_review','critic_review']) 




https://www.rottentomatoes.com/m/parasite_2019/reviews
https://www.rottentomatoes.com/m/parasite_2019/reviews?type=user
https://www.rottentomatoes.com/m/ford_v_ferrari_2019/reviews
https://www.rottentomatoes.com/m/ford_v_ferrari_2019/reviews?type=user
https://www.rottentomatoes.com/m/learning_to_skateboard_in_a_warzone_(if_you're_a_girl)_2019/reviews
https://www.rottentomatoes.com/m/learning_to_skateboard_in_a_warzone_(if_you're_a_girl)_2019/reviews?type=user
https://www.rottentomatoes.com/m/the_neighbors'_window_2019/reviews
https://www.rottentomatoes.com/m/the_neighbors'_window_2019/reviews?type=user
https://www.rottentomatoes.com/m/little_women_2019/reviews
https://www.rottentomatoes.com/m/little_women_2019/reviews?type=user
https://www.rottentomatoes.com/m/marriage_story_2019/reviews
https://www.rottentomatoes.com/m/marriage_story_2019/reviews?type=user
https://www.rottentomatoes.com/m/jojo_rabbit_2019/reviews
https://www.rottentomatoes.com/m/jojo_rabbit_2019/reviews?type=user
http

In [4]:
dfObj['critic_review']

0     [Good story, beautiful to watch and much to th...
1     [So emotional. Best ending of a movie I seen i...
2                                                    []
3                                                    []
4     [A good movie for wife and teenage kids %% , P...
5     [in a marriage story no one is bad and no one ...
6     [So great, funny and moving in one movie! %% ,...
7                                                    []
8     [It helps to know the Joker if you're a fan of...
9     [Best movie of the year. It was hilarious. Bes...
10    [The projector broke.  Show was cancelled. %% ...
11    [When first watching the beginning of the movi...
12    [Excellent acting and film %% , detailed side ...
13    [Instead of going on stage, Elton John permitt...
14    [American Factory is one of the greatest docum...
15                                                   []
Name: critic_review, dtype: object

In [5]:
dfObj

,Name,url_critics,url_audi,user_review,critic_review
0,parasite_2019,https://www.rottentomatoes.com/m/parasite_2019...,https://www.rottentomatoes.com/m/parasite_2019...,[ Parasite surprised me over and over. I const...,"[Good story, beautiful to watch and much to th..."
1,ford_v_ferrari_2019,https://www.rottentomatoes.com/m/ford_v_ferrar...,https://www.rottentomatoes.com/m/ford_v_ferrar...,[ ...it happens to be great filmmaking and fea...,[So emotional. Best ending of a movie I seen i...
2,learning_to_skateboard_in_a_warzone_(if_you're...,https://www.rottentomatoes.com/m/learning_to_s...,https://www.rottentomatoes.com/m/learning_to_s...,[],[]
3,the_neighbors'_window_2019,https://www.rottentomatoes.com/m/the_neighbors...,https://www.rottentomatoes.com/m/the_neighbors...,[],[]
4,little_women_2019,https://www.rottentomatoes.com/m/little_women_...,https://www.rottentomatoes.com/m/little_women_...,[ What's enjoyable about this version of Littl...,"[A good movie for wife and teenage kids %% , P..."
5,marriage_story_2019,https://www.rottentomatoes.com/m/marriage_stor...,https://www.rottentomatoes.com/m/marriage_stor...,[ Baumbach's approach to the challenge of repr...,[in a marriage story no one is bad and no one ...
6,jojo_rabbit_2019,https://www.rottentomatoes.com/m/jojo_rabbit_2...,https://www.rottentomatoes.com/m/jojo_rabbit_2...,"[ Playful, funny and relevant to the world we ...","[So great, funny and moving in one movie! %% ,..."
7,toy_story_4_2019,https://www.rottentomatoes.com/m/toy_story_4_2...,https://www.rottentomatoes.com/m/toy_story_4_2...,[],[]
8,joker_2019,https://www.rottentomatoes.com/m/joker_2019/re...,https://www.rottentomatoes.com/m/joker_2019/re...,[ It's disingenuous and pandering for Phillips...,[It helps to know the Joker if you're a fan of...
9,once_upon_a_time_in_hollywood_2019,https://www.rottentomatoes.com/m/once_upon_a_t...,https://www.rottentomatoes.com/m/once_upon_a_t...,[ This is often surprisingly funny in a playfu...,[Best movie of the year. It was hilarious. Bes...


In [0]:

# def get_user_reviews():
#     user_review=[]
#     for url in url_audi:
#       if '2019' in url:
#         print(url)
#         response = requests.get(url)

#         soup = BeautifulSoup(response.text, "html.parser")


#         # audience reviews
#         audience_reviews=soup.findAll('p', attrs={'class': 'audience-reviews__review--mobile js-review-text clamp clamp-4 js-clamp'})
#         if len(audience_reviews)!=0 :
#           for reviews in audience_reviews:
#             # print(reviews.text)
#             user_review.append(reviews.text+' %% ')
#     return user_review
      


      # def get_ctric_reviews():
#   critic_review=[]
#   review_pat = re.compile(r'<div class=\"the_review\">[;a-zA-Z\s,-.\'\/\?\[\]\":\']*</div>')


#   for url in url_critics:
#     if '2019' in url:
#       print(url)
#       response = requests.get(url)
#       soup = BeautifulSoup(response.text, "html.parser")

#       soup = str(soup)
#       if not len(soup.split('="review_table'))==1:
        
#         review_soup = soup.split('="review_table')[1].split('row review_table_row')
#         review_soup.pop(0)
#         review_soup = soup.split('="review_table')[1].split('row review_table_row')
#         review_soup.pop(0)
#         for review in review_soup:
                
#                 # extract review
#                 match = re.findall(review_pat, str(review))
#                 if len(match) > 0:
#                     m = match[0]
#                     for iden in ['<div class="the_review"> ','</div>']:
#                         m = m.replace(iden,'')
#                     reviews.append(m.strip('"'))
            

#         # if len(audience_reviews)!=0 :
#         for review in reviews:
#           if review.startswith('<div class="the_review">'):
#             critic_review.append(review.replace('<div class="the_review">','').replace('                                   ','').replace('\n','').replace('                                ','') + '$$')
#             # print(review.replace('<div class="the_review">','').replace('                                   ','').replace('\n',''))
#   return critic_review

In [111]:

response = requests.get('https://www.rottentomatoes.com/m/american_factory_2019/reviews')
crtc_review=[]
soup = BeautifulSoup(response.text, "html.parser")
page_no=soup.findAll('span', attrs={'class': 'pageInfo'})
page_no

num=int(page_no[0].text[-2:])
response = requests.get('https://www.rottentomatoes.com/m/american_factory_2019/reviews/')
crtc_review=[]
page_no=soup.findAll('span', attrs={'class': 'pageInfo'})
num=int(page_no[0].text[-2:])
print(num)
while(num!=0):
  response = requests.get('https://www.rottentomatoes.com/m/american_factory_2019/reviews?type=&sort=&page='+str(num))
  if response.status_code ==200:
    soup = BeautifulSoup(response.text, "html.parser")
    c_reviews=soup.findAll('div', attrs={'class': 'the_review'})
    # c_reviews=soup.findAll('div', attrs={'class': 'the_review'})
    page_no=soup.findAll('span', attrs={'class': 'pageInfo'})
    
    num-=1
    if len(c_reviews)!=0 :
      for reviews in c_reviews:
        crtc_review.append(reviews.text.replace('                                   ','').replace('\n','').replace('                                ','')+'$$' )
crtc_review
        

5


[' A fascinating tragicomedy about the incompatibility of American and Chinese industries.$$',
 ' [A] fascinating as-it-happens look at a cross-cultural clash in the industrialized world.$$',
 ' American Factory may not have the character-driven allure or celebrity factor of most successful theatrical docs, but it should be widely embraced by film festivals and TV buyers around the world...$$',
 " The film is too earnest and well-intentioned to fall prey to racism, but there's a vein of xenophobia baked into its premise that lingers and leaves more of a bad taste than it should.$$",
 " The filmmakers' probing analysis reveals the basic principles of freedom and dignity within the political essence of labor issues.$$",
 ' American Factory is going to be one of the most essential documentaries of the year.$$',
 ' If American Factory has any immediate takeaway, it is that regardless of where you are working in the world, there is always someone in power looking to exploit you$$',
 ' In th

5


[' A fascinating tragicomedy about the incompatibility of American and Chinese industries.$$',
 ' [A] fascinating as-it-happens look at a cross-cultural clash in the industrialized world.$$',
 ' American Factory may not have the character-driven allure or celebrity factor of most successful theatrical docs, but it should be widely embraced by film festivals and TV buyers around the world...$$',
 " The film is too earnest and well-intentioned to fall prey to racism, but there's a vein of xenophobia baked into its premise that lingers and leaves more of a bad taste than it should.$$",
 " The filmmakers' probing analysis reveals the basic principles of freedom and dignity within the political essence of labor issues.$$",
 ' American Factory is going to be one of the most essential documentaries of the year.$$',
 ' If American Factory has any immediate takeaway, it is that regardless of where you are working in the world, there is always someone in power looking to exploit you$$',
 ' In th

In [102]:

crtc_review

[<div class="the_review">
                                     The filmmaking does not advocate; it shows, never tells, and poses many more questions than it can answer. Some disturbing realities about the struggle for dignity in work emerge from what we witness.
                                 </div>, <div class="the_review">
                                     Could not have been more timely.
                                 </div>, <div class="the_review">
                                     Indeed, race relations seem to be a recurring theme, but one the filmmakers refuse to address head-on.
                                 </div>, <div class="the_review">
                                     Rarely does a sober, even-keeled reckoning come along to illuminate the human suffering of creative destruction while gently acknowledging that this is the way it has to be. Such a portrayal is American Factory, a heartbreaking Netflix documentary.
                                 </div>, <